# Проверенные идеи/ход решения

## Ideas without ML

Первая идея в том, что нужно искать однокоренные слова в `train` датасете и надеяться, что ударение не сместилось при добавлении/изменении приставки/суффикса/окончания.

1. Попробуем для каждого слова искать ближайшее к нему, где расстояние между словами определим как расстояние по Левенштайну. Чтобы алгоритм работал не слишком долго бин. поиском найдем в `train` первое слово, большее текущего (где текущее - то, для которого мы ищем ответ) и посмотрим ближайшие $K$ чисел, где $K$ можно выбрать 30 и будет нормально работать (потому что похожих слов (однкоренных, стоящих рядом) будет не более 30 обычно).
Это получает $89.87\%$. Может быть небольшая погрешность т.к. при равном расстоянии я меняю с шансом $1/3$.

2. Все тоже самое, но вместо Левенштайна будем считать по расстоянию Jaro–Winkler. Это получат примерно такой же результат ($89.28\%$).

3. Попробуем минимизировать такую формулу: $-10 \cdot lcp_i + levenstein_i$, где $lcp$ - наибольший общий префикс текущего слова и $train_i$, а $levenstein_i$ - расстояние по Левенштайну. $-10$ потому что хочется чтобы большой префикс был приоритетнее изменения каких-то символов в слове.
Это получает значительный прирост, точность - $92.27\%$.

4. Прошлое улучшение хорошо сработало, давайте попробуем смотреть не только на префикс, но еще и на наибольший корень. Его обозначим как наибольшую общую подстроку. Это не помогло и результат стал $91.12\%$.

5. Давайте попробуем для уже выбранного слова смотреть не просто на позицию ударения, а пытаться отбросить префиксы (приставки), перебирая их. Это выдало такой же результат.

Лучший результат - $92.27\%$.

---


Была идея попробовать кластеризацию, но после результата `RNN` я подумал, что это уже не нужно т.к. явно лучше не будет.

## CNN

Теперь попробуем написать что-то, основанное на нейросетях. Слова превратим в тензоры размера `<длина максимального слова x 1 x число букв>` используя one-hot-encoding. Длину максимального слова обрежем до $20$ вместо $35+$, что уменьшит число параметров у модели.


6. Попробуем сделать обычную `CNN`. Результат получился $10\%$, что примерно равно рандому.

7. Делать свертки размером $3 \times 3$ звучит не очень логично, не имеет какого-смысла. Попробуем сделать свертки по $k$ соседним буквам. *Также я немного увеличил Dropout и уменьшил полносвязные слои чтобы нейросеть не переобучалась так сильно (не помогло).* $9\%$...

Было ожидаемо, что свертки будут работать не очень, хотя от второго подхода я ожидал около $30-40%$. Возможно, стоило дольше обучать или попытаться сделать архитектуру получше, но это явно не будет лучше прошлых решений.

---

## RNN

Понятно, что свертки в этой задаче использовать не очень хорошая идея, поэтому давайте попробуем RNN, они умеют запоминать прошлые входные данные и скорее всего будут давать нормальный результат.

8. Сделаем `RNN` по символам. Для начала попробуем самую стандартную `RNN`. Она получает результат в $79.08\%$, что намного лучше, чем `CNN`, но недостаточно хорошо.
9. Вместо `nn.RNN()` можно попробовать `nn.GRU()`. Это выдает точность $91.62\%$, что уже близко к решению без мля.
10. Будем подбирать какие-то гиперпараметры, подробнее можно посмотреть сравнив 9 и 10 посылки. Получилось $95.84\%$.
11. Раз `mean` работает лучше `max`, то я подумал, что можно брать взвешенную сумму. $96.15\%$.
12. `LSTM` должен работать получше, чем `GRU`, давайте сделаем `Bidirectional-LSTM`. Это получает $96.92\%$.
13. После `RNN` на вход полносвязным слоям будем подавать не просто вывод `RNN` после агрегации, но еще сконкатенируем его с ембедингами. Также будем дообучать с меньшим `lr`. Это получает $97.4\%$.
14. Когда уже подобрана неплохая модель можно не делить датасет на обучающаю и валидационную выборки, а просто обучить модель на всем датасете. Скорее всего мне не повезло с обучением, лучшего результата не получилось.

На закрытом датасете модель получила результат получше - $97.79\%$.